In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from io import BytesIO
import cv2
import bson
from skimage.data import imread
import matplotlib.pyplot as plt
import keras
from PIL import Image
import io, bson, multiprocessing
import os
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
def get_the_data(path):
    data = bson.decode_file_iter(open(path, 'rb'))
    images=[]
    category=[]
    for c, d in enumerate(data):
        product_id = d['_id']
        category_id = d['category_id'] 
        for e, pic in enumerate(d['imgs']):
            category.append(category_id)
            im=pic['picture']
            images.append(Image.open(io.BytesIO(im)))
        if(len(set(category))==100):
            break
    return category, images

In [3]:
product_category_train,image_train=get_the_data('E:\\kaggle\\train.bson')

In [4]:
y, rev_labels = pd.factorize(product_category_train)

In [5]:
len(y)

383

In [6]:
from sklearn.utils import shuffle
im_train,lab_train=shuffle(image_train,y)
test_im=im_train[350:]
test_lab=lab_train[350:]
image_train=im_train[:350]
label_train=lab_train[:350]

In [7]:
len_train = len(image_train)
len_test = len(test_im)

In [9]:
len_cat = len(image_train)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = label_train[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\DA\\train'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = image_train[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((label_train[i])!=(label_train[i-1])):
                n = label_train[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\DA\\train'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = image_train[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((label_train[i])==(label_train[i-1])):
                  n = label_train[i]
                  #print('cat', n)
                  d = image_train[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

In [10]:
len_cat = len(test_im)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = test_lab[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\DA\\test'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = test_im[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((test_lab[i])!=(test_lab[i-1])):
                n = test_lab[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\DA\\test'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = test_im[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((test_lab[i])==(test_lab[i-1])):
                  n = test_lab[i]
                  #print('cat', n)
                  d = test_im[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

# KERAS VGG19


In [13]:
img_width, img_height = 180, 180
top_model_weights_path = 'bottleneck_fc_model_vgg19_aug.h5'
train_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\DA\\train'
validation_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\DA\\test'
nb_train_samples = len_train
nb_validation_samples = len_test
epochs = 50
batch_size = 1

In [14]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG19(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train_vgg19_aug.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation_vgg19_aug.npy', 'wb'),
            bottleneck_features_validation)

In [15]:
save_bottlebeck_features()

Found 350 images belonging to 95 classes.
Found 33 images belonging to 18 classes.


In [21]:
train_data = np.load(open('bottleneck_features_train_vgg19_aug.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('bottleneck_features_validation_vgg19_aug.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=10,batch_size=1)
model.save_weights(top_model_weights_path)

Train on 280 samples, validate on 70 samples
Epoch 1/10
280/280 [==============================] - 46s 163ms/step - loss: 11.7026 - acc: 0.2714 - val_loss: 9.9011 - val_acc: 0.3857
Epoch 2/10
280/280 [==============================] - 44s 157ms/step - loss: 11.6856 - acc: 0.2750 - val_loss: 9.9011 - val_acc: 0.3857
Epoch 3/10
280/280 [==============================] - 44s 159ms/step - loss: 11.6856 - acc: 0.2750 - val_loss: 9.9011 - val_acc: 0.3857
Epoch 4/10
280/280 [==============================] - 43s 154ms/step - loss: 11.6856 - acc: 0.2750 - val_loss: 9.9011 - val_acc: 0.3857
Epoch 5/10
280/280 [==============================] - 43s 155ms/step - loss: 11.6856 - acc: 0.2750 - val_loss: 9.9011 - val_acc: 0.3857
Epoch 6/10
280/280 [==============================] - 43s 153ms/step - loss: 11.6856 - acc: 0.2750 - val_loss: 9.9011 - val_acc: 0.3857
Epoch 7/10
280/280 [==============================] - 42s 151ms/step - loss: 11.6856 - acc: 0.2750 - val_loss: 9.9011 - val_acc: 0.3857
Epo

In [35]:
img_width, img_height = 180, 180
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model_vgg19_aug.h5'
train_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\DA\\train'
validation_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\DA\\test'
nb_train_samples = len_train
nb_validation_samples = len_test
epochs = 10
batch_size = 1

In [23]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(180,180,3))
print('Model loaded.')

Model loaded.


In [24]:
from keras import regularizers
t_model = Sequential()
t_model.add(Flatten(input_shape=base_model.output_shape[1:]))
t_model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
t_model.add(Dropout(0.1))
t_model.add(Dense(len(rev_labels), activation='softmax'))

In [25]:
t_model.load_weights(top_model_weights_path)

In [26]:
from keras.models import Model
model = Model(inputs=base_model.input, outputs=t_model(base_model.output))

In [27]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_3
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 sequential_8


In [28]:
for layer in model.layers[:14]:
    layer.trainable = False

In [29]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [30]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [31]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [32]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

Found 350 images belonging to 95 classes.


In [33]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

Found 33 images belonging to 18 classes.


In [36]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)

Epoch 1/10
 - 540s - loss: 22.3915 - acc: 0.2829 - val_loss: 22.5065 - val_acc: 0.1212
Epoch 2/10
 - 536s - loss: 21.3279 - acc: 0.3943 - val_loss: 21.9329 - val_acc: 0.2727
Epoch 3/10
 - 518s - loss: 20.3248 - acc: 0.5343 - val_loss: 22.0256 - val_acc: 0.3333
Epoch 4/10
 - 526s - loss: 19.4622 - acc: 0.6771 - val_loss: 22.1532 - val_acc: 0.3939
Epoch 5/10
 - 526s - loss: 18.6902 - acc: 0.7971 - val_loss: 22.3411 - val_acc: 0.3939
Epoch 6/10
 - 533s - loss: 18.2519 - acc: 0.8514 - val_loss: 21.9067 - val_acc: 0.3939
Epoch 7/10
 - 527s - loss: 17.7119 - acc: 0.9029 - val_loss: 22.2069 - val_acc: 0.3939
Epoch 8/10
 - 475s - loss: 17.4662 - acc: 0.9086 - val_loss: 23.0573 - val_acc: 0.3636
Epoch 9/10
 - 552s - loss: 16.9972 - acc: 0.9800 - val_loss: 22.9780 - val_acc: 0.2727
Epoch 10/10
 - 542s - loss: 16.7666 - acc: 0.9714 - val_loss: 23.0561 - val_acc: 0.2727
